In [1]:
import warnings; warnings.filterwarnings("ignore");
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
import sklearn.feature_extraction.text as sktf
from difflib import SequenceMatcher as seq_matcher
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from ngram import NGram
from sklearn import pipeline, grid_search
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.metrics import mean_squared_error, make_scorer
from fuzzywuzzy import fuzz
from nltk.stem.porter import *
import xgboost as xgb
import time

start_time = time.time()
stemmer = PorterStemmer()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
print ("Starting Data Loading... ") 
df_train = pd.read_csv('./train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('./test.csv', encoding="ISO-8859-1")
df_attr = pd.read_csv('./attributes.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv('./product_descriptions.csv', encoding="ISO-8859-1")
print("--- Files Loaded: %s minutes ---" % round(((time.time() - start_time)/60),2))

Starting Data Loading... 
--- Files Loaded: 0.59 minutes ---


In [4]:
print ("Starting Feature Building ... ") 
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
print('df_brand \n', df_brand.head())

df_col = df_attr[df_attr.name.str.contains("Color", na=False)][["product_uid", "value"]].rename(columns={"value": "color"}) 
df_col['color'] = df_col['color'].astype(str)
df_color = df_col.groupby('product_uid', as_index=False).agg(lambda x: " ".join(x))
print('df_color \n', df_color.head())

df_mat = df_attr[df_attr.name.str.contains("Material", na=False)][["product_uid", "value"]].rename(columns={"value": "material"}) 
df_mat['material'] = df_mat['material'].astype(str)
df_material = df_mat.groupby('product_uid', as_index=False).agg(lambda x: ' '.join(x))
print('df_material \n', df_material.head())

Starting Feature Building ... 
df_brand 
      product_uid                           brand
9       100001.0              Simpson Strong-Tie
37      100002.0  BEHR Premium Textured DeckOver
69      100003.0                        STERLING
93      100004.0                     Grape Solar
122     100005.0                           Delta
df_color 
    product_uid                      color
0     100002.0      Browns / Tans Tugboat
1     100003.0                White White
2     100005.0              Chrome Chrome
3     100006.0  Stainless Steel Stainless
4     100007.0                Black Black
df_material 
    product_uid                         material
0     100001.0                 Galvanized Steel
1     100003.0                        Composite
2     100007.0                          Plastic
3     100008.0                            Steel
4     100009.0  Medium Density Fiberboard (MDF)


In [5]:
num_train = df_train.shape[0]

stop_w = ['for', 'xbi', 'and', 'in', 'th','on','sku','with','what','from','that','less','er','ing'] #'electr','paint','pipe','light','kitchen','wood','outdoor','door','bathroom'
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}

In [6]:
def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

In [7]:
def ngram_similarity(data,col1,col2):
    cos=[]
    for i in range(len(data.id)):        
        st=data[col1][i]
        title=data[col2][i]   
        n = NGram(title.split(), key=lambda x:x[1])
        for s in st.split():
            n.search(s) 
                
        tfidf = sktf.TfidfVectorizer().fit_transform([st,title])
        c=((tfidf * tfidf.T).A)[0,1]     
        cos.append(c)                                   
    return cos

In [8]:
def dist_cosine(data,col1,col2):   
    cos=[]
    for i in range(len(data.id)):        
        st=data[col1][i]
        title=data[col2][i]        
        tfidf = sktf.TfidfVectorizer().fit_transform([st,title])
        c=((tfidf * tfidf.T).A)[0,1]     
        cos.append(c)                                   
    return cos 

In [9]:
def mean_dist(data,col1,col2):
    mean_edit_s_t=[]
    for i in range(len(data)):
        search=data[col1][i]
        title=data[col2][i]
        max_edit_s_t_arr=[]       
        for s in search.split(): 
                max_edit_s_t=[]                
                for t in title.split():                   
                    a=seq_matcher(None,s,t).ratio()
                    max_edit_s_t.append(a)
                max_edit_s_t_arr.append(max_edit_s_t)         
        l=0   
        for item in max_edit_s_t_arr:
                l=l+max(item)     
        mean_edit_s_t.append(l/len(max_edit_s_t_arr))            
    return mean_edit_s_t 

In [10]:
from fuzzywuzzy import fuzz
def fuzzy_partial_ratio(col1,col2):
    return fuzz.partial_ratio(col1,col2)

def fuzzy_token_sort_ratio(col1,col2):
    return fuzz.token_sort_ratio(col1,col2)

In [11]:
def str_stem(s): 
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(",","") #could be number / segment later
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        #s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        
        s = s.lower()
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        return s
    else:
        return "null"

## This function perfroms spell correction. 
## It may take other automatically engineered dictionary as an additional input.
def spell_correction(s, automatic_spell_check_dict={}):
    s = s.replace("craftsm,an","craftsman")        
    s = re.sub(r'depot.com/search=', '', s)
    s = re.sub(r'pilers,needlenose', 'pliers, needle nose', s)    
    
    s=s.replace("ttt","tt")    
    s=s.replace("lll","ll") 
    s=s.replace("nnn","nn") 
    s=s.replace("rrr","rr") 
    s=s.replace("sss","ss") 
    s=s.replace("zzz","zz")
    s=s.replace("ccc","cc")
    s=s.replace("eee","ee")

    s=s.replace("acccessories","accessories")

    s=re.sub(r'\bscott\b', 'scotts', s) #brand
    s=re.sub(r'\borgainzer\b', 'organizer', s)
    s=re.sub(r'\bshark bite\b', 'sharkbite',s)
    
    s=s.replace("hinges with pishinges with pins","hinges with pins")    
    s=s.replace("virtue usa","virtu usa")
    s=re.sub('outdoor(?=[a-rt-z])', 'outdoor ', s)
    s=re.sub(r'\bdim able\b',"dimmable", s) 
    s=re.sub(r'\blink able\b',"linkable", s)
    s=re.sub(r'\bm aple\b',"maple", s)
    s=s.replace("aire acondicionado", "air conditioner")
    s=s.replace("borsh in dishwasher", "bosch dishwasher")
    s=re.sub(r'\bapt size\b','appartment size', s)
    s=re.sub(r'\barm[e|o]r max\b','armormax', s)
    s=re.sub(r' ss ',' stainless steel ', s)
    s=re.sub(r'\bmay tag\b','maytag', s)
    s=re.sub(r'\bback blash\b','backsplash', s)
    s=re.sub(r'\bbum boo\b','bamboo', s)
    s=re.sub(r'(?<=[0-9] )but\b','btu', s)
    s=re.sub(r'\bcharbroi l\b','charbroil', s)
    s=re.sub(r'\bair cond[it]*\b','air conditioner', s)
    s=re.sub(r'\bscrew conn\b','screw connector', s)
    s=re.sub(r'\bblack decker\b','black and decker', s)
    s=re.sub(r'\bchristmas din\b','christmas dinosaur', s)
    s=re.sub(r'\bdoug fir\b','douglas fir', s)
    s=re.sub(r'\belephant ear\b','elephant ears', s)
    s=re.sub(r'\bt emp gauge\b','temperature gauge', s)
    s=re.sub(r'\bsika felx\b','sikaflex', s)
    s=re.sub(r'\bsquare d\b', 'squared', s)
    s=re.sub(r'\bbehring\b', 'behr', s)
    s=re.sub(r'\bcam\b', 'camera', s)
    s=re.sub(r'\bjuke box\b', 'jukebox', s)
    s=re.sub(r'\brust o leum\b', 'rust oleum', s)
    s=re.sub(r'\bx mas\b', 'christmas', s)
    s=re.sub(r'\bmeld wen\b', 'jeld wen', s)
    s=re.sub(r'\bg e\b', 'ge', s)
    s=re.sub(r'\bmirr edge\b', 'mirredge', s)
    s=re.sub(r'\bx ontrol\b', 'control', s)
    s=re.sub(r'\boutler s\b', 'outlets', s)
    s=re.sub(r'\bpeep hole', 'peephole', s)
    s=re.sub(r'\bwater pik\b', 'waterpik', s)
    s=re.sub(r'\bwaterpi k\b', 'waterpik', s)
    s=re.sub(r'\bplex[iy] glass\b', 'plexiglass', s)
    s=re.sub(r'\bsheet rock\b', 'sheetrock',s)
    s=re.sub(r'\bgen purp\b', 'general purpose',s)
    s=re.sub(r'\bquicker crete\b', 'quikrete',s)
    s=re.sub(r'\bref ridge\b', 'refrigerator',s)
    s=re.sub(r'\bshark bite\b', 'sharkbite',s)
    s=re.sub(r'\buni door\b', 'unidoor',s)
    s=re.sub(r'\bair tit\b','airtight', s)
    s=re.sub(r'\bde walt\b','dewalt', s)
    s=re.sub(r'\bwaterpi k\b','waterpik', s)
    s=re.sub(r'\bsaw za(ll|w)\b','sawzall', s)
    s=re.sub(r'\blg elec\b', 'lg', s)
    s=re.sub(r'\bhumming bird\b', 'hummingbird', s)
    s=re.sub(r'\bde ice(?=r|\b)', 'deice',s)  
    s=re.sub(r'\bliquid nail\b', 'liquid nails', s)  
    s=re.sub(r'\bdeck over\b','deckover', s)
    s=re.sub(r'\bcounter sink(?=s|\b)','countersink', s)
    s=re.sub(r'\bpipes line(?=s|\b)','pipeline', s)
    s=re.sub(r'\bbook case(?=s|\b)','bookcase', s)
    s=re.sub(r'\bwalkie talkie\b','2 pair radio', s)
    s=re.sub(r'(?<=^)ks\b', 'kwikset',s)
    s=re.sub('(?<=[0-9])[\ ]*ft(?=[a-z])', 'ft ', s)
    s=re.sub('(?<=[0-9])[\ ]*mm(?=[a-z])', 'mm ', s)
    s=re.sub('(?<=[0-9])[\ ]*cm(?=[a-z])', 'cm ', s)
    s=re.sub('(?<=[0-9])[\ ]*inch(es)*(?=[a-z])', 'in ', s)
    
    s=re.sub(r'(?<=[1-9]) pac\b', 'pack', s)
 
    s=re.sub(r'\bcfl bulbs\b', 'cfl light bulbs', s)
    s=re.sub(r' cfl(?=$)', ' cfl light bulb', s)
    s=re.sub(r'candelabra cfl 4 pack', 'candelabra cfl light bulb 4 pack', s)
    s=re.sub(r'\bthhn(?=$|\ [0-9]|\ [a-rtuvx-z])', 'thhn wire', s)
    s=re.sub(r'\bplay ground\b', 'playground',s)
    s=re.sub(r'\bemt\b', 'emt electrical metallic tube',s)
    s=re.sub(r'\boutdoor dining se\b', 'outdoor dining set',s)
    
     
    if "a/c" in s:
        if ('unit' in s) or ('frost' in s) or ('duct' in s) or ('filt' in s) or ('vent' in s) or ('clean' in s) or ('vent' in s) or ('portab' in s):
            s=s.replace("a/c","air conditioner")
        else:
            s=s.replace("a/c","ac")

   
    external_data_dict={'airvents': 'air vents', 
    'antivibration': 'anti vibration', 
    'autofeeder': 'auto feeder', 
    'backbrace': 'back brace', 
    'behroil': 'behr oil', 
    'behrwooden': 'behr wooden', 
    'brownswitch': 'brown switch', 
    'byefold': 'bifold', 
    'canapu': 'canopy', 
    'cleanerakline': 'cleaner alkaline',
    'colared': 'colored', 
    'comercialcarpet': 'commercial carpet', 
    'dcon': 'd con', 
    'doorsmoocher': 'door smoocher', 
    'dreme': 'dremel', 
    'ecobulb': 'eco bulb', 
    'fantdoors': 'fan doors', 
    'gallondrywall': 'gallon drywall', 
    'geotextile': 'geo textile', 
    'hallodoor': 'hallo door', 
    'heatgasget': 'heat gasket', 
    'ilumination': 'illumination', 
    'insol': 'insulation', 
    'instock': 'in stock', 
    'joisthangers': 'joist hangers', 
    'kalkey': 'kelkay', 
    'kohlerdrop': 'kohler drop', 
    'kti': 'kit', 
    'laminet': 'laminate', 
    'mandoors': 'main doors', 
    'mountspacesaver': 'mount space saver', 
    'reffridge': 'refrigerator', 
    'refrig': 'refrigerator', 
    'reliabilt': 'reliability', 
    'replaclacemt': 'replacement', 
    'searchgalvanized': 'search galvanized', 
    'seedeater': 'seed eater', 
    'showerstorage': 'shower storage', 
    'straitline': 'straight line', 
    'subpumps': 'sub pumps', 
    'thromastate': 'thermostat', 
    'topsealer': 'top sealer', 
    'underlay': 'underlayment',
    'vdk': 'bdk', 
    'wallprimer': 'wall primer', 
    'weedbgon': 'weed b gon', 
    'weedeaters': 'weed eaters', 
    'weedwacker': 'weed wacker', 
    'wesleyspruce': 'wesley spruce', 
    'worklite': 'work light'}
     
    for word in external_data_dict.keys():
        s=re.sub(r'\b'+word+r'\b',external_data_dict[word], s)
        
    ############ replace words from dict
    for word in automatic_spell_check_dict.keys():
        s=re.sub(r'\b'+word+r'\b',automatic_spell_check_dict[word], s)
           
    return s

##### end of dunction 'spell_correction'
############################################


### another replacement dict used independently
another_replacement_dict={"undercabinet": "under cabinet", 
"snowerblower": "snower blower", 
"mountreading": "mount reading", 
"zeroturn": "zero turn", 
"stemcartridge": "stem cartridge", 
"greecianmarble": "greecian marble", 
"outdoorfurniture": "outdoor furniture", 
"outdoorlounge": "outdoor lounge", 
"heaterconditioner": "heater conditioner", 
"heater/conditioner": "heater conditioner", 
"conditioner/heater": "conditioner heater", 
"airconditioner": "air conditioner", 
"snowbl": "snow bl", 
"plexigla": "plexi gla", 
"whirlpoolga": "whirlpool ga", 
"whirlpoolstainless": "whirlpool stainless", 
"sedgehamm": "sledge hamm", 
"childproof": "child proof", 
"flatbraces": "flat braces", 
"zmax": "z max", 
"gal vanized": "galvanized", 
"battery powere weedeater": "battery power weed eater", 
"shark bite": "sharkbite", 
"rigid saw": "ridgid saw", 
"black decke": "black and decker", 
"exteriorpaint": "exterior paint", 
"fuelpellets": "fuel pellet", 
"cabinetwithouttops": "cabinet without tops", 
"castiron": "cast iron", 
"pfistersaxton": "pfister saxton ", 
"splitbolt": "split bolt", 
"soundfroofing": "sound froofing", 
"cornershower": "corner shower", 
"stronglus": "strong lus", 
"shopvac": "shop vac", 
"shoplight": "shop light", 
"airconditioner": "air conditioner", 
"whirlpoolga": "whirlpool ga", 
"whirlpoolstainless": "whirlpool stainless", 
"snowblower": "snow blower", 
"plexigla": "plexi gla", 
"trashcan": "trash can", 
"mountspacesaver": "mount space saver", 
"undercounter": "under counter", 
"stairtreads": "stair tread", 
"techni soil": "technisoil", 
"in sulated": "insulated", 
"closet maid": "closetmaid", 
"we mo": "wemo", 
"weather tech": "weathertech", 
"weather vane": "weathervane", 
"versa tube": "versatube", 
"versa bond": "versabond", 
"in termatic": "intermatic", 
"therma cell": "thermacell", 
"tuff screen": "tuffscreen", 
"sani flo": "saniflo", 
"timber lok": "timberlok", 
"thresh hold": "threshold", 
"yardguard": "yardgard", 
"incyh": "in.", 
"diswasher": "dishwasher", 
"closetmade": "closetmaid", 
"repir": "repair", 
"handycap": "handicap", 
"toliet": "toilet", 
"conditionar": "conditioner", 
"aircondition": "air conditioner", 
"aircondiioner": "air conditioner", 
"comercialcarpet": "commercial carpet", 
"commercail": "commercial", 
"inyl": "vinyl", 
"vinal": "vinyl", 
"vynal": "vinyl", 
"vynik": "vinyl", 
"skill": "skil", 
"whirpool": "whirlpool", 
"glaciar": "glacier", 
"glacie": "glacier", 
"rheum": "rheem", 
"one+": "1", 
"toll": "tool", 
"ceadar": "cedar", 
"shelv": "shelf", 
"toillet": "toilet", 
"toiet": "toilet", 
"toilest": "toilet", 
"toitet": "toilet", 
"ktoilet": "toilet", 
"tiolet": "toilet", 
"tolet": "toilet", 
"eater": "heater", 
"robi": "ryobi", 
"robyi": "ryobi", 
"roybi": "ryobi", 
"rayobi": "ryobi", 
"riobi": "ryobi", 
"screww": "screw", 
"stailess": "stainless", 
"dor": "door", 
"vaccuum": "vacuum", 
"vacum": "vacuum", 
"vaccum": "vacuum", 
"vinal": "vinyl", 
"vynal": "vinyl", 
"vinli": "vinyl", 
"viyl": "vinyl", 
"vynil": "vinyl", 
"vlave": "valve", 
"vlve": "valve", 
"walll": "wall", 
"steal": "steel", 
"stell": "steel", 
"pcv": "pvc", 
"blub": "bulb", 
"ligt": "light", 
"bateri": "battery", 
"kolher": "kohler", 
"fame": "frame", 
"have": "haven", 
"acccessori": "accessory", 
"accecori": "accessory", 
"accesnt": "accessory", 
"accesor": "accessory", 
"accesori": "accessory", 
"accesorio": "accessory", 
"accessori": "accessory", 
"repac": "replacement", 
"repalc": "replacement", 
"repar": "repair", 
"repir": "repair", 
"replacemet": "replacement", 
"replacemetn": "replacement", 
"replacemtn": "replacement", 
"replaclacemt": "replacement", 
"replament": "replacement", 
"toliet": "toilet", 
"skill": "skil", 
"whirpool": "whirlpool", 
"stailess": "stainless", 
"stainlss": "stainless", 
"stainstess": "stainless", 
"jigsaww": "jig saw", 
"woodwen": "wood", 
"pywood": "plywood", 
"woodebn": "wood", 
"repellant": "repellent", 
"concret": "concrete", 
"windos": "window", 
"wndows": "window", 
"wndow": "window", 
"winow": "window", 
"caamera": "camera", 
"sitch": "switch", 
"doort": "door", 
"coller": "cooler", 
"flasheing": "flashing", 
"wiga": "wigan", 
"bathroon": "bath room", 
"sinl": "sink", 
"melimine": "melamine", 
"inyrtior": "interior", 
"tilw": "tile", 
"wheelbarow": "wheelbarrow", 
"pedistal": "pedestal", 
"submerciable": "submercible", 
"weldn": "weld", 
"contaner": "container", 
"webmo": "wemo", 
"genis": "genesis", 
"waxhers": "washer", 
"softners": "softener", 
"sofn": "softener", 
"connecter": "connector", 
"heather": "heater", 
"heatere": "heater", 
"electic": "electric", 
"quarteround": "quarter round", 
"bprder": "border", 
"pannels": "panel", 
"framelessmirror": "frameless mirror", 
"paneling": "panel", 
"controle": "control", 
"flurescent": "fluorescent", 
"flourescent": "fluorescent", 
"molding": "moulding", 
"lattiace": "lattice", 
"barackets": "bracket", 
"vintemp": "vinotemp", 
"vetical": "vertical", 
"verticle": "vertical", 
"vesel": "vessel", 
"versatiube": "versatube", 
"versabon": "versabond", 
"dampr": "damper", 
"vegtable": "vegetable", 
"plannter": "planter", 
"fictures": "fixture", 
"mirros": "mirror", 
"topped": "top", 
"preventor": "breaker", 
"traiter": "trailer", 
"ureka": "eureka", 
"uplihght": "uplight", 
"upholstry": "upholstery", 
"untique": "antique", 
"unsulation": "insulation", 
"unfinushed": "unfinished", 
"verathane": "varathane", 
"ventenatural": "vent natural", 
"shoer": "shower", 
"floorong": "flooring", 
"tsnkless": "tankless", 
"tresers": "dresers", 
"treate": "treated", 
"transparant": "transparent", 
"transormations": "transformation", 
"mast5er": "master", 
"anity": "vanity", 
"tomostat": "thermostat", 
"thromastate": "thermostat", 
"kphler": "kohler", 
"tji": "tpi", 
"cuter": "cutter", 
"medalions": "medallion", 
"tourches": "torch", 
"tighrner": "tightener", 
"thewall": "the wall", 
"thru": "through", 
"wayy": "way", 
"temping": "tamping", 
"outsde": "outdoor", 
"bulbsu": "bulb", 
"ligh": "light", 
"swivrl": "swivel", 
"switchplate": "switch plate", 
"swiss+tech": "swiss tech", 
"sweenys": "sweeney", 
"susbenders": "suspender", 
"cucbi": "cu", 
"gaqs": "gas", 
"structered": "structured", 
"knops": "knob", 
"adopter": "adapter", 
"patr": "part", 
"storeage": "storage", 
"venner": "veneer", 
"veneerstone": "veneer stone", 
"stm": "stem", 
"steqamers": "steamer", 
"latter": "ladder", 
"steele": "steel", 
"builco": "bilco", 
"panals": "panel", 
"grasa": "grass", 
"unners": "runner", 
"maogani": "maogany", 
"sinl": "sink", 
"grat": "grate", 
"showerheards": "shower head", 
"spunge": "sponge", 
"conroller": "controller", 
"cleanerm": "cleaner", 
"preiumer": "primer", 
"fertillzer": "fertilzer", 
"spectrazide": "spectracide", 
"spaonges": "sponge", 
"stoage": "storage", 
"sower": "shower", 
"solor": "solar", 
"sodering": "solder", 
"powerd": "powered", 
"lmapy": "lamp", 
"naturlas": "natural", 
"sodpstone": "soapstone", 
"punp": "pump", 
"blowerr": "blower", 
"medicn": "medicine", 
"slidein": "slide", 
"sjhelf": "shelf", 
"oard": "board", 
"singel": "single", 
"paintr": "paint", 
"silocoln": "silicon", 
"poinsetia": "poinsettia", 
"sammples": "sample", 
"sidelits": "sidelight", 
"nitch": "niche", 
"pendent": "pendant", 
"shopac": "shop vac", 
"shoipping": "shopping", 
"shelfa": "shelf", 
"cabi": "cabinet", 
"nails18": "nail", 
"dewaqlt": "dewalt", 
"barreir": "barrier", 
"ilumination": "illumination", 
"mortice": "mortise", 
"lumes": "lumen", 
"blakck": "black", 
"exterieur": "exterior", 
"expsnsion": "expansion", 
"air condit$": "air conditioner", 
"double pole type chf breaker": "double pole type ch breaker", 
"mast 5 er": "master", 
"toilet rak": "toilet rack", 
"govenore": "governor", 
"in wide": "in white", 
"shepard hook": "shepherd hook", 
"frost fee": "frost free", 
"kitchen aide": "kitchen aid", 
"saww horse": "saw horse", 
"weather striping": "weatherstripper", 
"'girls": "girl", 
"girl's": "girl", 
"girls'": "girl", 
"girls": "girl", 
"girlz": "girl", 
"boy's": "boy", 
"boys'": "boy", 
"boys": "boy", 
"men's": "man", 
"mens'": "man", 
"mens": "mam", 
"men": "man", 
"women's": "woman", 
"womens'": "woman", 
"womens": "woman", 
"women": "woman", 
"kid's": "kid", 
"kids'": "kid", 
"kids": "kid", 
"children's": "kid", 
"childrens'": "kid", 
"childrens": "kid", 
"children": "kid", 
"child": "kid", 
"bras": "bra", 
"bicycles": "bike", 
"bicycle": "bike", 
"bikes": "bike", 
"refridgerators": "fridge", 
"refrigerator": "fridge", 
"refrigirator": "fridge", 
"freezer": "fridge", 
"memories": "memory", 
"fragance": "perfume", 
"fragrance": "perfume", 
"cologne": "perfume", 
"anime": "animal", 
"assassinss": "assassin", 
"assassin's": "assassin", 
"assassins": "assassin", 
"bedspreads": "bedspread", 
"shoppe": "shop", 
"extenal": "external", 
"knives": "knife", 
"kitty's": "kitty", 
"levi's": "levi", 
"squared": "square", 
"rachel": "rachael", 
"rechargable": "rechargeable", 
"batteries": "battery", 
"seiko's": "seiko", 
"ounce": "oz"
}
#### end of anoter_replacent_dict


valuable_words_list=['tv','downrod', 'sillcock', 'shelving', 'luminaire', 'paracord', 'ducting', \
    'recyclamat', 'rebar', 'spackling', 'hoodie', 'placemat', 'innoculant', 'protectant', \
    'colorant', 'penetrant', 'attractant', 'bibb', 'nosing', 'subflooring', 'torchiere', 'thhn',\
    'lantern','epoxy','cloth','trim','adhesive','light','lights','saw','pad','polish','nose','stove',\
    'elbow','elbows','lamp','door','doors','pipe','bulb','wood','woods','wire','sink','hose','tile','bath','table','duct',\
    'windows','mesh','rug','rugs','shower','showers','wheels','fan','lock','rod','mirror','cabinet','shelves','paint',\
    'plier','pliers','set','screw','lever','bathtub','vacuum','nut', 'nipple','straw','saddle','pouch','underlayment',\
    'shade','top', 'bulb', 'bulbs', 'paint', 'oven', 'ranges', 'sharpie', 'shed', 'faucet',\
    'finish','microwave', 'can', 'nozzle', 'grabber', 'tub', 'angles','showerhead', 'dehumidifier', \
    'shelving', 'urinal', 'mdf']

not_so_valuable_words_list= ['aaa','off','impact','square','shelves','finish','ring','flood','dual','ball','cutter',\
'max','off','mat','allure','diamond','drive', 'edge','anchor','walls','universal','cat', 'dawn','ion','daylight',\
'roman', 'weed eater', 'restore', 'design', 'caddy', 'pole caddy', 'jet', 'classic', 'element', 'aqua',\
'terra', 'decora', 'ez', 'briggs', 'wedge', 'sunbrella',  'adorne', 'santa', 'bella', 'duck', 'hotpoint',\
'duck', 'tech', 'titan', 'powerwasher', 'cooper lighting', 'heritage', 'imperial', 'monster', 'peak', 
'bell', 'drive', 'trademark', 'toto', 'champion', 'shop vac', 'lava', 'jet', 'flood', \
'roman', 'duck', 'magic', 'allen', 'bunn', 'element', 'international', 'larson', 'tiki', 'titan', \
 'space saver', 'cutter', 'scotch', 'adorne', 'ball', 'sunbeam', 'fatmax', 'poulan', 'ring', 'sparkle', 'bissell', \
 'universal', 'paw', 'wedge', 'restore', 'daylight', 'edge', 'americana', 'wacker', 'cat', 'allure', 'bonnie plants', \
 'troy', 'impact', 'buffalo', 'adams', 'jasco', 'rapid dry', 'aaa', 'pole caddy', 'pac', 'seymour', 'mobil', \
 'mastercool', 'coca cola', 'timberline', 'classic', 'caddy', 'sentry', 'terrain', 'nautilus', 'precision', \
 'artisan', 'mural', 'game', 'royal', 'use', 'dawn', 'task', 'american line', 'sawtrax', 'solo', 'elements', \
 'summit', 'anchor', 'off', 'spruce', 'medina', 'shoulder dolly', 'brentwood', 'alex', 'wilkins', 'natural magic', \
 'kodiak', 'metro', 'shelter', 'centipede', 'imperial', 'cooper lighting', 'exide', 'bella', 'ez', 'decora', \
 'terra', 'design', 'diamond', 'mat', 'finish', 'tilex', 'rhino', 'crock pot', 'legend', 'leatherman', 'remove', \
 'architect series', 'greased lightning', 'castle', 'spirit', 'corian', 'peak', 'monster', 'heritage', 'powerwasher',\
 'reese', 'tech', 'santa', 'briggs', 'aqua', 'weed eater', 'ion', 'walls', 'max', 'dual', 'shelves', 'square',\
 'hickory', "vikrell", "e3", "pro series", "keeper", "coastal shower doors", 'cadet','church','gerber','glidden',\
 'cooper wiring devices', 'border blocks', 'commercial electric', 'pri','exteria','extreme', 'veranda',\
 'gorilla glue','gorilla','shark','wen']

not_so_valuable_words_list+=['free','height', 'width', 'depth', 'model','pcs', 'thick','pack','adhesive','steel','cordless', 'aaa' 'b', 'nm', 'hc', 'insulated','gll', 'nutmeg',\
    'pnl', 'sotc','withe','stainless','chrome','beige','max','acrylic', 'cognac', 'cherry', 'ivory','electric','fluorescent', 'recessed', 'matte',\
    'propane','sku','brushless','quartz','gfci','shut','sds','value','brown','white','black','red','green','yellow','blue','silver','pink',\
    'gray','gold','thw','medium','type','flush',"metaliks", 'metallic', 'amp','btu','gpf','pvc','mil','gcfi','plastic', 'vinyl','aaa',\
    'aluminum','brass','antique', 'brass','copper','nickel','satin','rubber','porcelain','hickory','marble','polyacrylic','golden','fiberglass',\
    'nylon','lmapy','maple','polyurethane','mahogany','enamel', 'enameled', 'linen','redwood', 'sku','oak','quart','abs','travertine', 'resin',\
    'birch','birchwood','zinc','pointe','polycarbonate', 'ash', 'wool', 'rockwool', 'teak','alder','frp','cellulose','abz', 'male', 'female', 'used',\
    'hepa','acc','keyless','aqg','arabesque','polyurethane', 'polyurethanes','ardex','armorguard','asb', 'motion','adorne','fatpack',\
    'fatmax','feet','ffgf','fgryblkg', 'douglas', 'fir', 'fleece','abba', 'nutri', 'thermal','thermoclear', 'heat', 'water', 'systemic',\
    'heatgasget', 'cool', 'fusion', 'awg', 'par', 'parabolic', 'tpi', 'pint', 'draining', 'rain', 'cost', 'costs', 'costa','ecostorage',
    'mtd', 'pass', 'emt', 'jeld', 'npt', 'sch', 'pvc', 'dusk', 'dawn', 'lathe','lows','pressure', 'round', 'series','impact', 'resistant','outdoor',\
    'off', 'sawall', 'elephant', 'ear', 'abb', 'baby', 'feedback', 'fastback','jumbo', 'flexlock', 'instant', 'natol', 'naples','florcant',\
    'canna','hammock', 'jrc', 'honeysuckle', 'honey', 'serrano','sequoia', 'amass', 'ashford', 'gal','gas', 'gasoline', 'compane','occupancy',\
    'home','bakeware', 'lite', 'lithium', 'golith','gxwh',  'wht', 'heirloom', 'marine', 'marietta', 'cambria', 'campane','birmingham',\
    'bellingham','chamois', 'chamomile', 'chaosaw', 'chanpayne', 'thats', 'urethane', 'champion', 'chann', 'mocha', 'bay', 'rough',\
    'undermount', 'price', 'prices', 'way', 'air', 'bazaar', 'broadway', 'driveway', 'sprayway', 'subway', 'flood', 'slate', 'wet',\
    'clean', 'tweed', 'weed', 'cub', 'barb', 'salem', 'sale', 'sales', 'slip', 'slim', 'gang', 'office', 'allure', 'bronze', 'banbury',\
    'tuscan','tuscany', 'refinishing', 'fleam','schedule', 'doeskin','destiny', 'mean', 'hide', 'bobbex', 'pdi', 'dpdt', 'tri', 'order',\
    'kamado','seahawks','weymouth', 'summit','tel','riddex', 'alick','alvin', 'ano', 'assy', 'grade', 'barranco', 'batte','banbury',\
    'mcmaster', 'carr', 'ccl', 'china', 'choc', 'colle', 'cothom', 'cucbi', 'cuv', 'cwg', 'cylander', 'cylinoid', 'dcf', 'number', 'ultra',\
    'diat','discon', 'disconnect', 'plantation', 'dpt', 'duomo', 'dupioni', 'eglimgton', 'egnighter','ert','euroloft', 'everready',\
    'felxfx', 'financing', 'fitt', 'fosle', 'footage', 'gpf','fro', 'genis', 'giga', 'glu', 'gpxtpnrf', 'size', 'hacr', 'hardw',\
    'hexagon', 'hire', 'hoo','number','cosm', 'kelston', 'kind', 'all', 'semi', 'gloss', 'lmi', 'luana', 'gdak', 'natol', 'oatu',\
    'oval', 'olinol', 'pdi','penticlea', 'portalino', 'racc', 'rads', 'renat', 'roc', 'lon', 'sendero', 'adora', 'sleave', 'swu',
    'tilde', 'cordoba', 'tuvpl','yel', 'acacia','mig','parties','alkaline','plexiglass', 'iii', 'watt']

In [12]:
def seg_words(str1, str2):
    str2 = str2.lower()
    str2 = re.sub("[^a-z0-9./]"," ", str2)
    str2 = [z for z in set(str2.split()) if len(z)>2]
    words = str1.lower().split(" ")
    s = []
    for word in words:
        if len(word)>3:
            s1 = []
            s1 += segmentit(word,str2,True)
            if len(s)>1:
                s += [z for z in s1 if z not in ['er','ing','s','less'] and len(z)>1]
            else:
                s.append(word)
        else:
            s.append(word)
    return (" ".join(s))

In [13]:
def segmentit(s, txt_arr, t):
    st = s
    r = []
    for j in range(len(s)):
        for word in txt_arr:
            if word == s[:-j]:
                r.append(s[:-j])
                #print(s[:-j],s[len(s)-j:])
                s=s[len(s)-j:]
                r += segmentit(s, txt_arr, False)
    if t:
        i = len(("").join(r))
        if not i==len(st):
            r.append(st[i:])
    return r

In [14]:
def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt


In [15]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)

In [16]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')
df_all = pd.merge(df_all, df_color, how='left', on='product_uid')
df_all = pd.merge(df_all, df_material, how='left', on='product_uid')

print('df_all \n', df_all.head())

df_all 
    id                                      product_title product_uid  \
0   2                  Simpson Strong-Tie 12-Gauge Angle      100001   
1   3                  Simpson Strong-Tie 12-Gauge Angle      100001   
2   9  BEHR Premium Textured DeckOver 1-gal. #SC-141 ...      100002   
3  16  Delta Vero 1-Handle Shower Only Faucet Trim Ki...      100005   
4  17  Delta Vero 1-Handle Shower Only Faucet Trim Ki...      100005   

   relevance         search_term  \
0       3.00       angle bracket   
1       2.50           l bracket   
2       3.00           deck over   
3       2.33    rain shower head   
4       2.67  shower only faucet   

                                 product_description  \
0  Not only do angles make joints stronger, they ...   
1  Not only do angles make joints stronger, they ...   
2  BEHR Premium Textured DECKOVER is an innovativ...   
3  Update your bathroom with the Delta Vero Singl...   
4  Update your bathroom with the Delta Vero Singl...   

    

In [17]:
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))
df_all['color'] = df_all['color'].astype(str).map(lambda x:str_stem(x))
df_all['material'] = df_all['material'].astype(str).map(lambda x:str_stem(x))
df_all['search_term'] = df_all['search_term'].map(lambda x:spell_correction(x))
print("--- Stemming: %s minutes ---" % round(((time.time() - start_time)/60),2))

df_all['search_term'] = df_all['search_term'].map(lambda x: " ".join(\
        [another_replacement_dict[w] if w in another_replacement_dict.keys() else w \
         for w in x.split()]))
df_all['product_title'] = df_all['product_title'].map(lambda x:spell_correction(x))
df_all['product_title'] = df_all['product_title'].map(lambda x: " ".join(\
        [another_replacement_dict[w] if w in another_replacement_dict.keys() else w \
         for w in x.split()]))
df_all['product_description'] = df_all['product_description'].map(lambda x:spell_correction(x))
df_all['product_description'] = df_all['product_description'].map(lambda x: " ".join(\
        [another_replacement_dict[w] if w in another_replacement_dict.keys() else w \
         for w in x.split()]))
df_all['brand'] = df_all['brand'].map(lambda x:spell_correction(x))
df_all['brand'] = df_all['brand'].map(lambda x: " ".join(\
        [another_replacement_dict[w] if w in another_replacement_dict.keys() else w \
         for w in x.split()]))
df_all['color'] = df_all['color'].map(lambda x:spell_correction(x))
df_all['color'] = df_all['color'].map(lambda x: " ".join(\
        [another_replacement_dict[w] if w in another_replacement_dict.keys() else w \
         for w in x.split()]))
df_all['material'] = df_all['material'].map(lambda x:spell_correction(x))
df_all['material'] = df_all['material'].map(lambda x: " ".join(\
        [another_replacement_dict[w] if w in another_replacement_dict.keys() else w \
         for w in x.split()]))
print("--- Replacement & Spelling Correction: %s minutes ---" % round(((time.time() - start_time)/60),2))

df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title']+"\t"+df_all['product_description']

print('df_all \n', df_all.head())

--- Stemming: 19.39 minutes ---
--- Replacement & Spelling Correction: 30.74 minutes ---
df_all 
    id                                      product_title product_uid  \
0   2                    simpson strong tie 12 gaug angl      100001   
1   3                    simpson strong tie 12 gaug angl      100001   
2   9  behr premium textur deckov 1gal. #sc 141 tugbo...      100002   
3  16  delta vero 1 handl shower onli faucet trim kit...      100005   
4  17  delta vero 1 handl shower onli faucet trim kit...      100005   

   relevance         search_term  \
0       3.00        angl bracket   
1       2.50           l bracket   
2       3.00            deckover   
3       2.33    rain shower head   
4       2.67  shower onli faucet   

                                 product_description  \
0  not onli do angl make joint stronger they also...   
1  not onli do angl make joint stronger they also...   
2  behr premium textur deckov is an innov solid c...   
3  updat your bathroom with 

In [18]:
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_color'] = df_all['color'].astype(str).map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_material'] = df_all['material'].astype(str).map(lambda x:len(x.split())).astype(np.int64)
print("--- Calculate Lenght: %s minutes ---" % round(((time.time() - start_time)/60),2))
print('df_all \n', df_all.head())

--- Calculate Lenght: 34.45 minutes ---
df_all 
    id                                      product_title product_uid  \
0   2                    simpson strong tie 12 gaug angl      100001   
1   3                    simpson strong tie 12 gaug angl      100001   
2   9  behr premium textur deckov 1gal. #sc 141 tugbo...      100002   
3  16  delta vero 1 handl shower onli faucet trim kit...      100005   
4  17  delta vero 1 handl shower onli faucet trim kit...      100005   

   relevance         search_term  \
0       3.00        angl bracket   
1       2.50           l bracket   
2       3.00            deckover   
3       2.33    rain shower head   
4       2.67  shower onli faucet   

                                 product_description  \
0  not onli do angl make joint stronger they also...   
1  not onli do angl make joint stronger they also...   
2  behr premium textur deckov is an innov solid c...   
3  updat your bathroom with the delta vero singl ...   
4  updat your bathroo

In [19]:
df_all['search_term'] = df_all['product_info'].map(lambda x:seg_words(x.split('\t')[0],x.split('\t')[1]))

df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))
print("--- Query in: %s minutes ---" % round(((time.time() - start_time)/60),2))

df_all['query_last_word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))
print("--- Query Last word in: %s minutes ---" % round(((time.time() - start_time)/60),2))

df_all["cosine_s.brand"]=dist_cosine(df_all,"search_term","brand")
df_all["cosine_s.material"]=dist_cosine(df_all,"search_term","material")
df_all["cosine_s.title"]=dist_cosine(df_all,"search_term","product_title")
print("--- Calculate Cosine Similarity: %s minutes ---" % round(((time.time() - start_time)/60),2))

df_all["mean_s.brand"]=mean_dist(df_all,"search_term","brand")
df_all["mean_s.material"]=mean_dist(df_all,"search_term","material")
df_all["mean_s.title"]=mean_dist(df_all,"search_term","product_title")
print("--- Calculate Mean: %s minutes ---" % round(((time.time() - start_time)/60),2))

print('df_all \n', df_all.head())

--- Query in: 34.69 minutes ---
--- Query Last word in: 34.71 minutes ---
--- Calculate Cosine Similarity: 53.53 minutes ---
--- Calculate Mean: 57.46 minutes ---
df_all 
    id                                      product_title product_uid  \
0   2                    simpson strong tie 12 gaug angl      100001   
1   3                    simpson strong tie 12 gaug angl      100001   
2   9  behr premium textur deckov 1gal. #sc 141 tugbo...      100002   
3  16  delta vero 1 handl shower onli faucet trim kit...      100005   
4  17  delta vero 1 handl shower onli faucet trim kit...      100005   

   relevance         search_term  \
0       3.00        angl bracket   
1       2.50           l bracket   
2       3.00            deckover   
3       2.33    rain shower head   
4       2.67  shower onli faucet   

                                 product_description  \
0  not onli do angl make joint stronger they also...   
1  not onli do angl make joint stronger they also...   
2  behr pr

In [20]:
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']+"\t"+df_all['color'].astype(str)+"\t"+df_all['material'].astype(str)
df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_color'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['word_in_material'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[3]))
df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']
df_all['ratio_color'] = df_all['word_in_color']/df_all['len_of_color']
df_all['ratio_material'] = df_all['word_in_material']/df_all['len_of_material']
print("--- Calculate Common Word & Ratio: %s minutes ---" % round(((time.time() - start_time)/60),2))

print('df_all \n', df_all.head())

--- Calculate Common Word & Ratio: 57.52 minutes ---
df_all 
    id                                      product_title product_uid  \
0   2                    simpson strong tie 12 gaug angl      100001   
1   3                    simpson strong tie 12 gaug angl      100001   
2   9  behr premium textur deckov 1gal. #sc 141 tugbo...      100002   
3  16  delta vero 1 handl shower onli faucet trim kit...      100005   
4  17  delta vero 1 handl shower onli faucet trim kit...      100005   

   relevance         search_term  \
0       3.00        angl bracket   
1       2.50           l bracket   
2       3.00            deckover   
3       2.33    rain shower head   
4       2.67  shower onli faucet   

                                 product_description  \
0  not onli do angl make joint stronger they also...   
1  not onli do angl make joint stronger they also...   
2  behr premium textur deckov is an innov solid c...   
3  updat your bathroom with the delta vero singl ...   
4  updat

In [21]:
### 1. Fuzzy partial_ratio
### 2. Fuzzy token_sort_ratio
df_all['fuzzy_token_sort_ratio_in_title'] = df_all['product_info'].map(lambda x:fuzzy_token_sort_ratio(x.split('\t')[0],x.split('\t')[1]))
df_all['fuzzy_token_sort_ratio_in_description'] = df_all['product_info'].map(lambda x:fuzzy_token_sort_ratio(x.split('\t')[0],x.split('\t')[2]))

df_all['fuzzy_token_sort_ratio_in_brand'] = df_all['attr'].map(lambda x:fuzzy_token_sort_ratio(x.split('\t')[0],x.split('\t')[1]))
df_all['fuzzy_token_sort_ratio_in_color'] = df_all['attr'].map(lambda x:fuzzy_token_sort_ratio(x.split('\t')[0],x.split('\t')[2]))
df_all['fuzzy_token_sort_ratio_in_material'] = df_all['attr'].map(lambda x:fuzzy_token_sort_ratio(x.split('\t')[0],x.split('\t')[3]))

df_all['fuzzy_token_partial_ratio_in_title'] = df_all['product_info'].map(lambda x:fuzzy_partial_ratio(x.split('\t')[0],x.split('\t')[1]))
df_all['fuzzy_token_partial_ratio_in_description'] = df_all['product_info'].map(lambda x:fuzzy_partial_ratio(x.split('\t')[0],x.split('\t')[2]))

df_all['fuzzy_token_partial_ratio_in_brand'] = df_all['attr'].map(lambda x:fuzzy_partial_ratio(x.split('\t')[0],x.split('\t')[1]))
df_all['fuzzy_token_partial_ratio_in_color'] = df_all['attr'].map(lambda x:fuzzy_partial_ratio(x.split('\t')[0],x.split('\t')[2]))
df_all['fuzzy_token_partial_ratio_in_material'] = df_all['attr'].map(lambda x:fuzzy_partial_ratio(x.split('\t')[0],x.split('\t')[3]))
print("--- Calculate Fuzzy Ratio: %s minutes ---" % round(((time.time() - start_time)/60),2))

df_all.head()

--- Calculate Fuzzy Ratio: 58.95 minutes ---


,id,product_title,product_uid,relevance,search_term,product_description,brand,color,material,product_info,...,fuzzy_token_sort_ratio_in_title,fuzzy_token_sort_ratio_in_description,fuzzy_token_sort_ratio_in_brand,fuzzy_token_sort_ratio_in_color,fuzzy_token_sort_ratio_in_material,fuzzy_token_partial_ratio_in_title,fuzzy_token_partial_ratio_in_description,fuzzy_token_partial_ratio_in_brand,fuzzy_token_partial_ratio_in_color,fuzzy_token_partial_ratio_in_material
0,2,simpson strong tie 12 gaug angl,100001,3.00,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,nan,galvan steel,angl bracket\tsimpson strong tie 12 gaug angl\...,...,33,3,27,27,33,33,67,33,33,33
1,3,simpson strong tie 12 gaug angl,100001,2.50,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,nan,galvan steel,l bracket\tsimpson strong tie 12 gaug angl\tno...,...,15,2,15,17,29,33,44,33,33,33
2,9,behr premium textur deckov 1gal. #sc 141 tugbo...,100002,3.00,deckover,behr premium textur deckov is an innov solid c...,behr premium textur deckov,brown tan tugboat,nan,deckover\tbehr premium textur deckov 1gal. #sc...,...,21,2,47,8,0,75,75,86,12,0
3,16,delta vero 1 handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero singl ...,delta,chrome chrome,nan,rain shower head\tdelta vero 1 handl shower on...,...,31,5,19,41,21,62,69,40,46,33
4,17,delta vero 1 handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero singl ...,delta,chrome chrome,nan,shower onli faucet\tdelta vero 1 handl shower ...,...,39,6,17,39,19,100,72,40,38,33


In [22]:
df_brand = pd.unique(df_all.brand.ravel())
d={}
i = 1000
for s in df_brand:
    d[s]=i
    i+=3

df_all['brand_feature'] = df_all['brand'].map(lambda x:d[x])
df_all['search_term_feature'] = df_all['search_term'].map(lambda x:len(x))

print('df_all \n', df_all.head())
df_all.to_csv('df_all-1.csv')

df_all = df_all.drop(['search_term','product_title','product_description','product_info','brand', 'color', 'material', 'attr'],axis=1)

print('df_all \n', df_all.head())
df_all.to_csv('df_all-2.csv')


df_all 
    id                                      product_title product_uid  \
0   2                    simpson strong tie 12 gaug angl      100001   
1   3                    simpson strong tie 12 gaug angl      100001   
2   9  behr premium textur deckov 1gal. #sc 141 tugbo...      100002   
3  16  delta vero 1 handl shower onli faucet trim kit...      100005   
4  17  delta vero 1 handl shower onli faucet trim kit...      100005   

   relevance         search_term  \
0       3.00        angl bracket   
1       2.50           l bracket   
2       3.00            deckover   
3       2.33    rain shower head   
4       2.67  shower onli faucet   

                                 product_description  \
0  not onli do angl make joint stronger they also...   
1  not onli do angl make joint stronger they also...   
2  behr premium textur deckov is an innov solid c...   
3  updat your bathroom with the delta vero singl ...   
4  updat your bathroom with the delta vero singl ...   

    

In [23]:
print ("Starting Model Training... ") 
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']

y_train = df_train['relevance'].values
X_train = df_train.drop(['id','relevance'],axis=1).values
X_test = df_test.drop(['id','relevance'],axis=1).values

Starting Model Training... 


In [36]:
df_train.to_csv('df_train.csv')
df_test.to_csv('df_test.csv')

In [24]:
########################## RandomForestRegressor #################################

clf = RandomForestRegressor(n_estimators = 500, n_jobs = -1, random_state = 2016, verbose = 1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[y_pred > 3] = 3
y_pred[y_pred < 1] = 1
yo_pred = clf.predict(X_train)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_V9_RFR.csv',index=False)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 364.6min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    3.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    8.7s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    9.8s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    3.8s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    4.3s finished


In [25]:
########################## SVR #################################

clf = SVR(C=1.0, epsilon=0.2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[y_pred > 3] = 3
y_pred[y_pred < 1] = 1
yo_pred = clf.predict(X_train)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_V9_SVR.csv',index=False)

In [26]:
########################## LinearRegression #################################

clf = linear_model.LinearRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[y_pred > 3] = 3
y_pred[y_pred < 1] = 1
yo_pred = clf.predict(X_train)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_V9_GLR.csv',index=False)

In [27]:
########################## GradientBoostingRegressor #################################

clf = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss='ls')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[y_pred > 3] = 3
y_pred[y_pred < 1] = 1
yo_pred = clf.predict(X_train)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_V9_GBR.csv',index=False)

In [28]:
clf = xgb.XGBRegressor(max_depth = 6, n_estimators = 100)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[y_pred > 3] = 3
y_pred[y_pred < 1] = 1
yo_pred = clf.predict(X_train)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_V9_XGB.csv',index=False)

In [29]:
clf = KNeighborsRegressor()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[y_pred > 3] = 3
y_pred[y_pred < 1] = 1
yo_pred = clf.predict(X_train)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_V9_KNeighborsRegressor.csv',index=False)

In [34]:
clf = SGDRegressor()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[y_pred > 3] = 3
y_pred[y_pred < 1] = 1
yo_pred = clf.predict(X_train)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_V9_SGDRegressor.csv',index=False)

In [32]:
xgb = xgb.XGBRegressor(max_depth = 6, n_estimators = 100)
clf = BaggingRegressor(xgb, n_estimators=45, max_samples=0.1, random_state=25)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[y_pred > 3] = 3
y_pred[y_pred < 1] = 1
yo_pred = clf.predict(X_train)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_V9_XGB_Bagging.csv',index=False)

In [33]:
rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[y_pred > 3] = 3
y_pred[y_pred < 1] = 1
yo_pred = clf.predict(X_train)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_V9_RF_Bagging.csv',index=False)

In [35]:
clf = MLPRegressor()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[y_pred > 3] = 3
y_pred[y_pred < 1] = 1
yo_pred = clf.predict(X_train)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_V9_MLP.csv',index=False)